In [23]:
import rasterio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import heapq
# from sklearn.metrics import confusion_matrix
# # from sklearn.metrics import precision_recall_curve
# from sklearn.metrics import precision_score
# from sklearn.metrics import average_precision_score
# from sklearn.metrics import recall_score
# from sklearn.metrics import f1_score 
# from sklearn.metrics import roc_curve
# from sklearn.metrics import auc
from sklearn.metrics import multilabel_confusion_matrix
# from skimage.filters import threshold_otsu
# similarity_case_1_p = '../../../results/RF/simliarity_measure/case_1_ref_mask_similarity_measure.tif'
# similarity_case_2_p = '../../../results/RF/simliarity_measure/case_2_ref_mask_similarity_measure.tif'
# similarity_case_3_p = '../../../results/RF/simliarity_measure/case_3_ref_mask_similarity_measure.tif'
# binary_change_map = '../../../results/RF/binary_change_D/change_map_case_1.tif'
# similarity_change_map = '../../../results/RF/simliarity_measure/optimal_threshold/sim-change_map_case_1.tif'
source_proba_distr = '../../../results/RF/simliarity_measure/2018_rf_model_1.npy'
# target_proba_distri = '../../../results/RF/simliarity_measure/2019_rf_model_1.npy'

gt_source_path  = '../../../data/rasterized_samples/2018_rasterizedImage.tif'
# gt_target_path = '../../../data/rasterized_samples/2019_rasterizedImage.tif'
# pred_source_path = '../../../results/RF/2018_rf_case_2_map.tif'
# pred_target_path = '../../../results/RF/2019_rf_case_3_map.tif'

In [2]:
source_proba = np.load(source_proba_distr)

In [3]:
source_proba = np.sort(source_proba, axis=1)[:, ::-1]

In [4]:
source_proba

array([[0.2154273 , 0.16797907, 0.12088084, ..., 0.0115641 , 0.00269574,
        0.00205739],
       [0.18054258, 0.17856504, 0.12668672, ..., 0.01568175, 0.00323386,
        0.00269574],
       [0.2257703 , 0.14244967, 0.08704204, ..., 0.00510253, 0.00360707,
        0.00301844],
       ...,
       [0.97664903, 0.01751358, 0.00188406, ..., 0.        , 0.        ,
        0.        ],
       [0.96421647, 0.02934249, 0.0022334 , ..., 0.        , 0.        ,
        0.        ],
       [0.95563428, 0.03953535, 0.00224455, ..., 0.        , 0.        ,
        0.        ]])

In [26]:
uncertainty = source_proba[:,0] - source_proba[:,1] 

In [24]:
with rasterio.open(gt_source_path) as src:
    width, height = src.shape
    profile = src.profile
    profile['nodata'] = -999.

In [27]:
uncertainty =uncertainty.reshape(height, width)

In [22]:
import os
model_name = os.path.basename(source_proba_distr).split('.')[-2]
# model_name = model_name.split('_')[0]
# model_name = '_'.join(model_name)

In [21]:
model_name

'npy'

In [7]:
def certainty(data, raster):
    probability_distr = np.load(data)
    probability_distr =np.sort(probability_distr, axis=1)[:,::-1]
    uncertainty = probability_distr[:,0] - probability_distr[:,1]
    with rasterio.open(raster) as src:
        profile = src.profile
        profile['nodata'] = -999.
        height, width = src.shape
    base_name = os.path.basename(data).split('.')[-2]
    base_name = base_name.split('_')[0]
    uncertainty = np.reshape(width, height)
    with rasterio.open(os.path.join(outdir, base_name + 'certainty' + case +'.tif'), 'w', **profile) as dst:
        dst.write(uncertainty, 1)
if __name__ == '__main__':
    _path  = '../../../data/rasterized_samples/2018_rasterizedImage.tif'
    for case in ['1', '2', '3']:
        proba_distr = '../../../results/RF/simliarity_measure/2018_rf_model_'+case+'.npy'
        certainty(proba_distr, _path)

array([0.04744823, 0.00197754, 0.08332063, ..., 0.95913544, 0.93487399,
       0.91609893])

In [2]:
gt_source_ = rasterio.open(gt_source_path).read(1).flatten().astype('int')
gt_target_ = rasterio.open(gt_target_path).read(1).flatten().astype('int')
pred_source_ = rasterio.open(pred_source_path).read(1).flatten()
pred_target_ = rasterio.open(pred_target_path).read(1).flatten()

In [3]:
gt_mask_nodata = (gt_source_ != 0) & (gt_target_ != 0)

In [4]:
gt_mask_nodata

array([False, False, False, ..., False, False, False])

In [5]:
def mask_and_extract(array, mask_array):
    array = np.ma.masked_array(array, mask=True)
    array.mask[mask_array] = False
    array_value = np.ma.compressed(array)
    return array_value
gt_source_ = mask_and_extract(gt_source_, gt_mask_nodata)
gt_target_ = mask_and_extract(gt_target_, gt_mask_nodata)
pred_source_ = mask_and_extract(pred_source_, gt_mask_nodata)
pred_target_ = mask_and_extract(pred_target_, gt_mask_nodata)

In [17]:
mcm = multilabel_confusion_matrix(df['gt_change'], df['pred_change'])

In [19]:
mcm

array([[[3883116,    2551],
        [   7765,  128288]],

       [[4021702,      18],
        [      0,       0]],

       [[4021617,     103],
        [      0,       0]],

       ...,

       [[4021399,     310],
        [     11,       0]],

       [[4021711,       9],
        [      0,       0]],

       [[3997454,     976],
        [  12878,   10412]]])

In [6]:
df = pd.DataFrame({'gt_source': gt_source_, 'gt_target': gt_target_, 'pred_source': pred_source_, 'pred_target': pred_target_})

In [7]:
dict_={1: 'Dense built-up area',
2: 'Diffuse built-up area',
 3: 'Industrial and commercial areas',
 4: 'Roads',
 5: 'Oilseeds (Rapeseed)',
 6: 'Straw cereals (Wheat, Triticale, Barley)',
 7: 'Protein crops (Beans / Peas)',
 8: 'Soy',
 9: 'Sunflower',
 10: 'Corn',
 12: 'Tubers/roots',
 13: 'Grasslands',
 14: 'Orchards and fruit growing',
 15: 'Vineyards',
 16: 'Hardwood forest',
 17: 'Softwood forest',
 18: 'Natural grasslands and pastures',
 19: 'Woody moorlands',
 23: 'Water'}

In [8]:
import time
start_time =time.time()
label = ['Dense built-up area', 'Diffuse built-up area', 'Industrial and commercial areas', 'Roads', 'Oilseeds (Rapeseed)', 'Straw cereals (Wheat, Triticale, Barley)', 'Protein crops (Beans / Peas)', 'Soy', 'Sunflower', 'Corn',  'Tubers/roots', 'Grasslands', 'Orchards and fruit growing', 'Vineyards', 'Hardwood forest', 'Softwood forest', 'Natural grasslands and pastures', 'Woody moorlands', 'Water']
df['gt_source'] = df['gt_source'].map(dict_)
df['gt_target'] = df['gt_target'].map(dict_)
df['pred_source'] = df['pred_source'].map(dict_)
df['pred_target'] = df['pred_target'].map(dict_)
print(" %s minute" % str((time.time() - start_time)/60))

 0.016994539896647134 minute


In [9]:
import time
start_time =time.time()
df['gt_change']= df.apply(lambda x: str(x['gt_source']) + '-' + str(x['gt_target']), axis=1)
df['pred_change']= df.apply(lambda x: str(x['pred_source']) + '-' + str(x['pred_target']), axis=1)
print(" %s minutes" % str((time.time() - start_time)/60))

 6.176694925626119 minutes


In [10]:
cm = pd.crosstab(df['gt_change'], df['pred_change'], rownames=['Ground Truth'], colnames=['Prediction'])

In [13]:
cm_recall = cm.apply(lambda x: x/x.sum(), axis=1)

In [14]:
cm_recall

Prediction,Corn-Corn,Corn-Diffuse built-up area,Corn-Grasslands,Corn-Industrial and commercial areas,Corn-Natural grasslands and pastures,Corn-Oilseeds (Rapeseed),Corn-Orchards and fruit growing,Corn-Protein crops (Beans / Peas),Corn-Roads,Corn-Soy,...,Woody moorlands-Industrial and commercial areas,Woody moorlands-Natural grasslands and pastures,Woody moorlands-Oilseeds (Rapeseed),Woody moorlands-Orchards and fruit growing,Woody moorlands-Softwood forest,Woody moorlands-Soy,"Woody moorlands-Straw cereals (Wheat, Triticale, Barley)",Woody moorlands-Sunflower,Woody moorlands-Vineyards,Woody moorlands-Woody moorlands
Ground Truth,,,,,,,,,,,,,,,,,,,,,
Corn-Corn,0.942927,0.000051,0.000235,0.000118,0.0,0.000162,0.0,0.000015,0.0,0.002198,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
Corn-Oilseeds (Rapeseed),0.000000,0.000000,0.000000,0.000000,0.0,0.846682,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
Corn-Protein crops (Beans / Peas),0.000000,0.000000,0.000000,0.000939,0.0,0.015962,0.0,0.839437,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
Corn-Soy,0.007666,0.000000,0.000299,0.000381,0.0,0.000000,0.0,0.000000,0.0,0.896806,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
"Corn-Straw cereals (Wheat, Triticale, Barley)",0.000052,0.000000,0.000000,0.000515,0.0,0.000155,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woody moorlands-Oilseeds (Rapeseed),0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
Woody moorlands-Soy,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
"Woody moorlands-Straw cereals (Wheat, Triticale, Barley)",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.002985,0.000000,0.000000,0.000000


In [17]:
cm_recall('cm.csv')

In [7]:
# df['gt_change']= df.apply(lambda x: str(x['gt_source']) + '-' + str(x['gt_target']), axis=1)
# df['pred_change']= df.apply(lambda x: str(x['pred_source']) + '-' + str(x['pred_target']), axis=1)

In [ ]:
def changeErrorCheck(
    case,
    gt_source,
    gt_target,
    pred_source,
    pred_target,
    outdir
    ):
    # read rasters
    gt_source_ = rasterio.open(gt_source).read(1).flatten().astype('int')
    gt_target_ = rasterio.open(gt_target).read(1).flatten().astype('int')
    pred_source_ = rasterio.open(pred_source).read(1).flatten()
    pred_target_ = rasterio.open(pred_target).read(1).flatten()
    
    gt_mask_nodata = (gt_source_ != 0) & (gt_target_ != 0)
    
    def mask_and_extract(array, mask_array):
        array = np.ma.masked_array(array, mask=True)
        array.mask[mask_array] = False
        array_value = np.ma.compressed(array)
        return array_value

    gt_source_ = mask_and_extract(gt_source_, gt_mask_nodata)
    gt_target_ = mask_and_extract(gt_target_, gt_mask_nodata)
    pred_source_ = mask_and_extract(pred_source_, gt_mask_nodata)
    pred_target_ = mask_and_extract(pred_target_, gt_mask_nodata)
    
    # class label
    dict_={1: 'Dense built-up area',
    2: 'Diffuse built-up area',
     3: 'Industrial and commercial areas',
     4: 'Roads',
     5: 'Oilseeds (Rapeseed)',
     6: 'Straw cereals (Wheat, Triticale, Barley)',
     7: 'Protein crops (Beans / Peas)',
     8: 'Soy',
     9: 'Sunflower',
     10: 'Corn',
     12: 'Tubers/roots',
     13: 'Grasslands',
     14: 'Orchards and fruit growing',
     15: 'Vineyards',
     16: 'Hardwood forest',
     17: 'Softwood forest',
     18: 'Natural grasslands and pastures',
     19: 'Woody moorlands',
     23: 'Water'}
    
    df = pd.DataFrame({'gt_source': gt_source_, 'gt_target': gt_target_, 'pred_source': pred_source_, 'pred_target': pred_target_})
    # rename class to label
    df['gt_source'] = df['gt_source'].map(dict_)
    df['gt_target'] = df['gt_target'].map(dict_)
    df['pred_source'] = df['pred_source'].map(dict_)
    df['pred_target'] = df['pred_target'].map(dict_)
    
    
    df['gt_change']= df.apply(lambda x: str(x['gt_source']) + '-' + str(x['gt_target']), axis=1)
    df['pred_change']= df.apply(lambda x: str(x['pred_source']) + '-' + str(x['pred_target']), axis=1)
    
    cm = pd.crosstab(df['gt_change'], df['pred_change'], rownames=['Ground Truth'], colnames=['Prediction'])
    
    cm.to_csv("semantic_change_matrix" + case + '.csv')

if __name__ == '__main__':
    gt_source_path = '../../../data/rasterized_samples/2018_rasterizedImage.tif'
    gt_target_path = '../../../data/rasterized_samples/2019_rasterizedImage.tif'
    
    for case in ['1', '2', '3']:
        pred_source_path = '../../../results/RF/2018_rf_case_'+ case +'_map.tif'
        pred_target_path = '../../../results/RF/2019_rf_case_'+ case +'_map.tif'
        outdir_ = '../../../results/RF/binary_change_D'
        changeErrorCheck(case, gt_source_path, gt_target_path, pred_source_path, pred_target_path, outdir)